In [ ]:
"""
g_wave_setup.ipynb

Created by Chris Stevens 2023
Copyright (c) 2023 University of Canterbury. All rights reserved.
"""

################################################################################
# Import required libraries
################################################################################

import numpy as np
import h5py
from mpi4py import MPI

from coffee import ibvp, actions, solvers, grid
from coffee.diffop.sbp import sbp

import g_wave
import g_wave_plotter_2 as gw_plotter

np.set_printoptions(threshold=np.inf, precision=16)

In [ ]:
################################################################################
# Simulation parameters
################################################################################

N      = 200   # Number of sptial intervals (number of points  - 1)
a      = 32.   # Amplitude of the waves
A      = 4.    # A function related to the scale of the spatial direction
pl     = [0.]  # An array of polarisations for the wave(s) travelling rightward
pr     = [0.]  # An array of polarisations for the wave(s) travelling leftward

tau    = 1.    # A parameter for the SAT method (greater than or equal to 1)
CFL    = 0.2   # The Courant-Friedrichs-Lewy number (less than or equal to 1)

zstart = -1.   # Lower bound for grid
zstop  = 1.    # Lower bound for grid
tstart = 0.    # Start of simulation
tstop  = 1.    # Stop of simulation

store_output = False      # Output HDF5 data
filename     = str(N) + '_order2.hdf'  # Name of HDF5 file

display_output = True     # Display animation during simulation

In [ ]:
################################################################################
# Instantiate the operator which approximates spatial derivatives
################################################################################

# Second-order finite difference operator
diffop = sbp.D21_CNG(sbp.BOUNDARY_TYPE_GHOST_POINTS)

# Fourth-order finite difference operator
# diffop = sbp.D43_Strand(sbp.BOUNDARY_TYPE_GHOST_POINTS)

In [ ]:
################################################################################
# MPI setup
################################################################################

dims = MPI.Compute_dims(MPI.COMM_WORLD.size, [0])                                    
periods = [0]                                                                        
reorder = True                                                                       
mpi_comm = MPI.COMM_WORLD.Create_cart(dims, periods=periods, reorder=reorder)        


In [ ]:
################################################################################
# Grid setup
################################################################################

ghost_points = (diffop.ghost_points(),)
internal_points = (diffop.internal_points(),)
b_data = grid.MPIBoundary(
    ghost_points, 
    internal_points, 
    mpi_comm=mpi_comm, 
    number_of_dimensions=1
)

grid = grid.UniformCart(
        (N,), 
        [[zstart,zstop]],
        comparison = N,
        mpi_comm = mpi_comm,
        boundary_data=b_data
    )

global_z = np.linspace(zstart, zstop, N+1)

In [ ]:
################################################################################
# Instantiate the system
################################################################################

system = g_wave.G_wave(\
        diffop, tau, global_z, 
        CFL = CFL, 
        amplitude = a,
        A=A,
        pl = pl, pr = pr
        )

In [ ]:
################################################################################
# Instantiate the solver
################################################################################

solver = solvers.RungeKutta4(system)
maxIteration = 10000000

In [ ]:
################################################################################
# Details for file output when necessary
################################################################################

if store_output and mpi_comm.rank == 0:
    hdf_file = h5py.File(filename, "w")
    output_actions = [
    actions.SimOutput.Data(),
    actions.SimOutput.Times(),
    actions.SimOutput.Domains(),
    actions.SimOutput.Constraints()
    ]

In [ ]:
################################################################################
# Instantiate the IBVP problem
################################################################################
   
actionList = []
if store_output and mpi_comm.rank == 0:
    actionList += [actions.SimOutput(\
        hdf_file,\
        solver, \
        system, \
        grid, \
        output_actions,\
        overwrite = True,\
        name = grid.name,\
        cmp_ = grid.comparison\
        )];
if display_output and mpi_comm.rank == 0:
    actionList += [gw_plotter.Plotter(
        system,
        frequency = 10, 
        xlim = (zstart, zstop),
        ylim = [-a*0.1, a*1.1], 
        findex = (7,8,3,4,5,6),
        labels = (r"$\Psi_0$",r"$\Psi_4$",r"$\rho$",r"$\rho$'", \
                    r"$\sigma$",r"$\sigma'$"),
        delay = 0.0001
    )]
problem = ibvp.IBVP(solver, system, grid = grid,\
        maxIteration = 1000000, action = actionList,\
        minTimestep = 1e-6)

################################################################################
# Run the simulation
################################################################################

problem.run(tstart, tstop)

if store_output and mpi_comm.rank == 0:
    hdf_file.close()